In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60103737.dfs.core.windows.net",
    "9QM9wDRIg06Z0Udsjzr40x7DPmmPA0sjcarGLXFCxGuuBc/a1AbneCE0g5GXbbnWvLDSSM3Z2jYT+AStO3eiSA=="
)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, lower, length, regexp_replace

# Create session 
spark = SparkSession.builder.appName("GoodreadsTextFeatures").getOrCreate()

df = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/gold/features_v1/"
)

df = df.filter(col("review_text").isNotNull() & (length(trim(col("review_text"))) >= 10))
df = df.filter(col("rating").isNotNull())


df = df.withColumn("review_text", lower(trim(col("review_text"))))
df = df.withColumn("review_text", regexp_replace(col("review_text"), r"http\S+", "url"))
df = df.withColumn("review_text", regexp_replace(col("review_text"), r"\d+", "number"))
df = df.withColumn("review_text", regexp_replace(col("review_text"), r"[^\w\s]", ""))
df = df.withColumn("review_text", regexp_replace(col("review_text"), r"\s+", " "))

df = df.select("review_id", "book_id", "user_id", "rating", "review_text")

df.show(5, truncate=False)
df.printSchema()


+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:

train_df, val_df, test_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)

print("Train:", train_df.count())
print("Validation:", val_df.count())
print("Test:", test_df.count())


Train: 10481260
Validation: 2244923
Test: 2245188


In [0]:
from pyspark.sql.functions import lower, trim, regexp_replace, col, length


train_df = train_df.withColumn("clean_text",
    regexp_replace(lower(trim(col("review_text"))), r"http\S+", "url")
)
train_df = train_df.withColumn("clean_text",
    regexp_replace(col("clean_text"), r"\d+", "number")
)
train_df = train_df.withColumn("clean_text",
    regexp_replace(col("clean_text"), r"[^\w\s]", "")
)
train_df = train_df.withColumn("clean_text",
    regexp_replace(col("clean_text"), r"\s+", " ")
)

# Filter very short reviews
train_df = train_df.filter(length(col("clean_text")) >= 10)

train_df.select("review_text", "clean_text").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import size, split

train_df = train_df.withColumn("review_length_words", size(split(col("clean_text"), " ")))
train_df = train_df.withColumn("review_length_chars", length(col("clean_text")))

train_df.select("review_length_words", "review_length_chars").describe().show()


+-------+-------------------+-------------------+
|summary|review_length_words|review_length_chars|
+-------+-------------------+-------------------+
|  count|           10434470|           10434470|
|   mean| 129.77496125821435|  703.6296581426752|
| stddev| 180.83020124961234|  992.2496408726851|
|    min|                  1|                 10|
|    max|               5009|              31856|
+-------+-------------------+-------------------+



In [0]:

train_sample = train_df.limit(50000)

pdf = train_sample.toPandas()

print("Sample size:", len(pdf))
pdf.head(3)


Sample size: 50000


,review_id,book_id,user_id,rating,review_text,clean_text,review_length_words,review_length_chars
0,0000660f0dc7b670ecf9254ecc14674f,11735983,b48f60be86aca1257c2cc4cdf5a6a67d,5,although i didnt think it was as good as diver...,although i didnt think it was as good as diver...,379,2040
1,0001228ce8aa9c181cc5fbf9eb438dd8,25981019,1f01674bff3f2a30cb870a6c928b4875,4,elizabet wakes up in hospital with no memory o...,elizabet wakes up in hospital with no memory o...,137,739
2,000134fb82ff548d2781d0aaabc2a8db,25550525,dfc964adb7c9e481c45c88c407bbbfe8,3,xo blake this story was a very nice surprise a...,xo blake this story was a very nice surprise a...,49,256


In [0]:

!pip install nltk


import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

sentiments = pdf["clean_text"].apply(lambda x: sia.polarity_scores(str(x)))

pdf["sentiment_pos"] = sentiments.apply(lambda x: x["pos"])
pdf["sentiment_neg"] = sentiments.apply(lambda x: x["neg"])
pdf["sentiment_neu"] = sentiments.apply(lambda x: x["neu"])
pdf["sentiment_compound"] = sentiments.apply(lambda x: x["compound"])

pdf[["clean_text", "sentiment_compound"]].head(3)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 20.5/40.5 kB 741.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 691.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 345.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 51.2/57.7 kB 1.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.8 MB/s eta 0:00:01
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 3.0 MB/s eta 0:00:01
   ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/1.5 MB 1.8 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.5 MB/s eta 0:00:01
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/1.5 MB 2.2 MB/s eta 0:00:01
   ━

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-131f76e7-5039-4742-9d0d-51/nltk_data...


,clean_text,sentiment_compound
0,although i didnt think it was as good as diver...,-0.8425
1,elizabet wakes up in hospital with no memory o...,0.2043
2,xo blake this story was a very nice surprise a...,0.6518


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf = TfidfVectorizer(max_features=1000, stop_words='english', ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(pdf["clean_text"])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df.head(3)


,ability,able,absolutely,action,actions,actual,actually,add,added,addition,admit,adult,adventure,afraid,age,ago,al,alert,alex,alive,alla,alpha,amazing,america,american,anche,angst,annoying,apart,appreciate,arc,arent,art,ask,aspect,attention,attraction,auch,author,authors,...,women,wonder,wonderful,wont,word,words,work,worked,working,works,world,worse,worth,wouldnt,wow,write,writer,writes,writing,writing style,written,wrong,wrote,ya,yang,yeah,year,year old,years,years ago,yes,yg,yn,york,youll,young,younger,youre,youve,zu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095075,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
train_final = pd.concat([
    pdf[["review_id", "book_id", "rating", "review_length_words", "review_length_chars",
         "sentiment_pos", "sentiment_neg", "sentiment_neu", "sentiment_compound"]],
    tfidf_df,
], axis=1)

train_final.head(3)


,review_id,book_id,rating,review_length_words,review_length_chars,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound,ability,able,absolutely,action,actions,actual,actually,add,added,addition,admit,adult,adventure,afraid,age,ago,al,alert,alex,alive,alla,alpha,amazing,america,american,anche,angst,annoying,apart,appreciate,arc,...,women,wonder,wonderful,wont,word,words,work,worked,working,works,world,worse,worth,wouldnt,wow,write,writer,writes,writing,writing style,written,wrong,wrote,ya,yang,yeah,year,year old,years,years ago,yes,yg,yn,york,youll,young,younger,youre,youve,zu
0,0000660f0dc7b670ecf9254ecc14674f,11735983,5,379,2040,0.115,0.138,0.747,-0.8425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0001228ce8aa9c181cc5fbf9eb438dd8,25981019,4,137,739,0.087,0.068,0.845,0.2043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000134fb82ff548d2781d0aaabc2a8db,25550525,3,49,256,0.195,0.054,0.751,0.6518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:

print(train_final.columns[train_final.columns.duplicated()])

train_final = train_final.loc[:, ~train_final.columns.duplicated()]

train_final.columns = (
    train_final.columns
    .str.replace('[ ,;{}()\n\t=]', '_', regex=True)  
    .str.strip('_')                                  
    .str.replace('__+', '_', regex=True)             
    .str.lower()                                      
)

print("Cleaned column : ")



Index(['rating'], dtype='object')
Cleaned column : 


In [0]:
train_final = train_final.loc[:, ~train_final.columns.duplicated()]


In [0]:
train_spark_df = spark.createDataFrame(train_final)

train_spark_df = train_spark_df.repartition(8)

train_spark_df.write.mode("overwrite").format("delta").save(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/gold/features_v2/train_features/"
)
print("Train Delta table successfully written!")


Train Delta table successfully written!
